In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azure.storage.blob import BlockBlobService
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.65


In [2]:
ws = Workspace.from_config()

project_folder = './player_status'
os.makedirs(project_folder, exist_ok=True)

experiment_name = 'player_status'
experiment = Experiment(ws, name=experiment_name)

In [8]:
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                      datastore_name='playerstatus', 
                                                      container_name='playerstatus',
                                                      account_name='omnicintellige6390360867', 
                                                      account_key='Z1u6QIYTp8QOP+ECSUcyCtNk+zBLBoq7jdsyvG64fbVsv/0MBzlHxFKNObVB3MVn9MZE2rlVQJXSv5TaBw4dHQ==')

In [4]:
ds = Datastore.get(ws, 'omnicintellige6390360867')

In [25]:
block_blob_service = BlockBlobService(account_name='omnicintellige6390360867', account_key='Z1u6QIYTp8QOP+ECSUcyCtNk+zBLBoq7jdsyvG64fbVsv/0MBzlHxFKNObVB3MVn9MZE2rlVQJXSv5TaBw4dHQ==')

container_name ='playerstatus'
generator = block_blob_service.list_blobs(container_name)
datastore_paths = []
for blob in generator:
    datastore_paths.append((datastore, blob.name))

status_ds = Dataset.File.from_files(path=datastore_paths)

In [26]:
print(status_ds)
status_ds = status_ds.register(workspace = ws,
                                 name = 'player_status_ds',
                                 description = 'Test of player status',
                                 create_new_version = True)

FileDataset
{
  "source": [
    "('playerstatus', '10021.hdf5')",
    "('playerstatus', '10035.hdf5')",
    "('playerstatus', '10050.hdf5')",
    "('playerstatus', '10090.hdf5')",
    "('playerstatus', '10092.hdf5')",
    "('playerstatus', '10093.hdf5')",
    "('playerstatus', '7015.hdf5')",
    "('playerstatus', '7016.hdf5')",
    "('playerstatus', '7017.hdf5')",
    "('playerstatus', '7018.hdf5')",
    "('playerstatus', '7019.hdf5')",
    "('playerstatus', '7022.hdf5')",
    "('playerstatus', '7023.hdf5')",
    "('playerstatus', '7024.hdf5')",
    "('playerstatus', '7025.hdf5')",
    "('playerstatus', '7026.hdf5')",
    "('playerstatus', '7088.hdf5')",
    "('playerstatus', '7089.hdf5')",
    "('playerstatus', '7090.hdf5')",
    "('playerstatus', '7091.hdf5')",
    "('playerstatus', '7319.hdf5')",
    "('playerstatus', '7326.hdf5')",
    "('playerstatus', '7327.hdf5')",
    "('playerstatus', '7328.hdf5')",
    "('playerstatus', '7329.hdf5')",
    "('playerstatus', '7330.hdf5')",
    

In [24]:
status_ds.to_path()

array(['/playerstatus/7015.hdf5', '/playerstatus/alive_set.txt',
       '/playerstatus/antiheal_set.txt', '/playerstatus/color_set.txt',
       '/playerstatus/hero_set.txt', '/playerstatus/immortal_set.txt',
       '/playerstatus/rounds.txt', '/playerstatus/side_set.txt',
       '/playerstatus/spectator_mode_set.txt',
       '/playerstatus/status_set.txt', '/playerstatus/switch_set.txt',
       '/playerstatus/ult_set.txt'], dtype=object)

In [12]:
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [37]:

script_params = {
    '--data_folder': status_ds.as_named_input('status').as_mount(),
    '--beta': 0.5,
    '--num_files': 0,
    '--learning_rate': 0.001,
    '--num_epochs': 20,
    '--early_stopping': 2,
    '--recent': True,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='pytorch_train.py',
                    use_gpu=True,
                    pip_packages=['h5py', 'azureml-sdk',
                                  'azureml-dataprep[pandas,fuse]>=1.1.14'])

WARNING - framework_version is not specified, defaulting to version 1.2.


In [38]:
run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)

RunId: player_status_1571110238_98d3e7a7
Web View: https://mlworkspace.azure.ai/portal/subscriptions/1d1428b3-3300-4696-99ad-a1df55934195/resourceGroups/omnic_intelligence/providers/Microsoft.MachineLearningServices/workspaces/omnic_intelligence/experiments/player_status/runs/player_status_1571110238_98d3e7a7

Streaming azureml-logs/55_azureml-execution-tvmps_c26c6ca18f40c5175d00fa21389fffbf1a1912f070f7ef4151cd98c05f2bb99d_d.txt

2019-10-15T03:30:55Z Successfully mounted a/an Azure File Shares at /mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_status_1571110238_98d3e7a7/mounts/workspacefilestore
2019-10-15T03:30:55Z Mounted //omnicintellige6390360867.file.core.windows.net/azureml-filestore-6085e42c-59ef-4f3b-a949-7099435c60e9 at /mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_status_1571110238_98d3e7a7/mounts/workspacefilestore
2019-10-15T03:30:55Z No blob file systems configured
2019-10-15T03:30:55Z No unmanaged file systems configured
201

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')"))': /azureml/ExperimentRun/dcid.player_status_1571110238_98d3e7a7/azureml-logs/70_driver_log.txt?sv=2018-11-09&sr=b&sig=AhmQarBiFwNxTdvRlI0N1y5zidTVMcbyVaO2AKCpao0%3D&st=2019-10-15T05%3A42%3A35Z&se=2019-10-15T13%3A52%3A35Z&sp=r


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "ServiceError",
        "message": "Dataset finalization failed: [Errno 12] Cannot allocate memory",
        "details": [],
        "debugInfo": {
            "type": "OSError",
            "message": "[Errno 12] Cannot allocate memory",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_status_1571110238_98d3e7a7/mounts/workspacefilestore/azureml/player_status_1571110238_98d3e7a7/azureml-setup/context_manager_injector.py\", line 49, in __exit__\n    self.context_manager.__exit__(*exc_details)\n  File \"/mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_status_1571110238_98d3e7a7/mounts/workspacefilestore/azureml/player_status_1571110238_98d3e7a7/azureml-setup/context_managers.py\", line 214, in __exit__\n    self.datasets.__exit__(*args)\n  File \"/azureml-envs/azureml_0e740a9d917dda255b36b061dc181deb/

In [ ]:
model = run.register_model(model_name='player_status_cnn', model_path='outputs/')

In [ ]:
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
PyTorch.get_supported_versions()
